In [1]:
import pandas as pd
import numpy as np

In [2]:
# 深证成指日对数收益率计算
SZCZ = pd.read_csv('C:/Users/jxjsj/Desktop/JupyterHome/Data/SZCZ080101-190131.csv',index_col=0)

SZCZ_p = pd.DataFrame(SZCZ['close'],index=SZCZ.index)

SZCZ_p['close-1'] = SZCZ_p['close'].shift(1)

SZCZ_lnR = pd.DataFrame(np.log(SZCZ_p['close'])-np.log(SZCZ_p['close-1']),index=SZCZ.index,columns=['lnR']).dropna(axis=0)

SZCZ_lnR_2 = pd.DataFrame(SZCZ_lnR['lnR'].apply(lambda x:x*x))

SZCZ_lnR_2.columns = ['lnR_2']

pd.concat([SZCZ_lnR,SZCZ_lnR_2],axis=1).to_csv('C:/Users/jxjsj/Desktop/JupyterHome/Data/SZCZ_r_new.csv')

In [3]:
SZCZ_r = pd.concat([SZCZ_lnR,SZCZ_lnR_2],axis=1)

In [4]:
# 深证成指已实现波动率计算
df_fminSZCZ = pd.read_csv(r'C:\Users\jxjsj\Desktop\JupyterHome\Data\SZCZ_five_min080101-190131.csv',index_col=0,header=0)

df_fminSZCZ['time'] = list(df_fminSZCZ.index)

df_fminSZCZ['date'] = list(df_fminSZCZ['time'].apply(lambda x: str(x)[:10]))

df_fminSZCZ

df_fminSZCZ['close-1'] = df_fminSZCZ['close'].shift(1)

df_fminSZCZ_lnR_temp = pd.DataFrame(np.log(df_fminSZCZ['close'])-np.log(df_fminSZCZ['close-1']),index=df_fminSZCZ.index,columns=['lnR'])

df_fminSZCZ_lnR = pd.concat([df_fminSZCZ_lnR_temp,pd.DataFrame(list(df_fminSZCZ['date']),columns=['date'],index=df_fminSZCZ.index)],axis=1).dropna(axis=0,how='any')

df_fminSZCZ_lnR['lnR^2'] = df_fminSZCZ_lnR['lnR'].apply(lambda x: x*x)

groupby_temp = df_fminSZCZ_lnR['lnR^2'].groupby(df_fminSZCZ_lnR['date'])

groupby_df = pd.DataFrame(groupby_temp.sum())

rvSZCZ = pd.DataFrame(list(groupby_temp.sum()),columns=['SZCZRV'],index=groupby_df.index)

In [5]:
df_temp1 = pd.merge(SZCZ_r,rvSZCZ, left_index=True,right_index=True)

In [6]:
# 月份哑变量设计
time_temp = pd.date_range('1/1/2008','12/31/2019')

time_df = pd.DataFrame(time_temp,columns=['date'])

month_lst = list(time_df['date'].apply(lambda x: str(x)[5:7]))

month_df = pd.DataFrame(month_lst,columns=['month'])

month_temp = pd.get_dummies(month_df)

time_month_temp = pd.concat([time_df,month_temp],axis=1)

time_month_temp.columns = ['date','Jan','Feb','Mar','Apr','May','June','July','Aug','Sep','Oct','Nov','Dec']

time_month = time_month_temp[['Jan','Feb','Mar','Apr','May','June','July','Aug','Sep','Oct','Nov','Dec']]

time_month.index=list(time_month_temp['date'])

In [7]:
df_temp2 = pd.merge(df_temp1,time_month, left_index=True,right_index=True)

In [8]:
# 星期哑变量设计
time_temp = pd.date_range('1/1/2008','12/31/2019')

time_df = pd.DataFrame(time_temp,columns=['date'])

lst_week = []
n=627
while n>0:
    for i in [2,3,4,5,6,7,1]:
        lst_week.append(i)
    n-=1

week_temp = pd.get_dummies(pd.DataFrame(lst_week[:4383],columns=['week'])['week'])

time_week_temp = pd.concat([time_df,week_temp],axis=1)

time_week_temp.columns = ['date','M','Tu','W','Th',"F","Sa",'Su']

time_week = time_week_temp[['M','Tu','W','Th',"F","Sa",'Su']]

time_week.index=list(time_week_temp['date'])

In [9]:
df_temp3 = pd.merge(df_temp2,time_week, left_index=True,right_index=True)

In [10]:
# 休市天数计算
import datetime
notrade = pd.DataFrame(list(SZCZ.index),columns=['date'])

notrade['date-1'] = notrade['date'].shift(1)

notrade = notrade.dropna(axis=0,how='any')

notrade_daycnt_temp = pd.DataFrame(notrade['date']+notrade['date-1'],columns=['daycnt'])

notrade_daycnt_lst = list(
    notrade_daycnt_temp['daycnt'].apply(
    lambda x: (datetime.datetime.strptime(x[:10], "%Y-%m-%d")-datetime.datetime.strptime(x[10:20], "%Y-%m-%d")).days-1
                         ))

notrade_daycnt = pd.DataFrame(notrade_daycnt_lst,columns=['ntd_cnt'])
notrade_daycnt.index = list(notrade['date'])

In [11]:
df_temp4 = pd.merge(df_temp3,notrade_daycnt, left_index=True,right_index=True)

In [12]:
# 换手率交易量
SZCZ_motion = pd.read_csv('C:/Users/jxjsj/Desktop/JupyterHome/Data/SZCZ_motion090101-190227.csv',index_col=0)

SZCZ_motion['volume-1'] = SZCZ_motion['volume'].shift(1)
SZCZ_motion['volumeR'] = np.log(SZCZ_motion['volume']) - np.log(SZCZ_motion['volume-1'])

SZCZ_motion['money-1'] = SZCZ_motion['money'].shift(1)
SZCZ_motion['moneyR'] = np.log(SZCZ_motion['money']) - np.log(SZCZ_motion['money-1'])

SZCZ_motion['turnoverR'] = SZCZ_motion['turnover_ratio']/100

SZCZ_motion_temp = SZCZ_motion[['volume','money','volumeR','moneyR','turnoverR']]

In [13]:
df_temp5 = pd.merge(df_temp4,SZCZ_motion_temp,how='left' ,left_index=True,right_index=True)
df_temp5 = df_temp5.fillna(0)

In [14]:
# 前日新闻条数
SZCZ_new = pd.read_csv('C:/Users/jxjsj/Desktop/JupyterHome/Data/news_cnt_090101-190201.csv',index_col=0)

df_temp6 = pd.merge(df_temp5,SZCZ_new,how='left' ,left_index=True,right_index=True)

In [15]:
# 5日均收益率
df_temp6['lnRroll5'] = df_temp6['lnR'].rolling(5).mean()

# 5日均换手率
df_temp6['turnoverRroll5'] = df_temp6['turnoverR'].rolling(5).mean()

# 20日均收益率
df_temp6['lnRroll20'] = df_temp6['lnR'].rolling(20).mean()

# 20日标准差
df_temp6['stdlnRroll20'] = df_temp6['lnR'].rolling(20).std()

# 20日夏普 无风险0.04
df_temp6['sharpe20'] = (df_temp6['lnRroll20']*250 - 0.04) / (df_temp6['stdlnRroll20']*(250**0.5))

# 60日偏度
df_temp6['skewlnR60'] = df_temp6['lnR'].rolling(60).skew()

# 60日峰度
df_temp6['kurtlnR60'] = df_temp6['lnR'].rolling(60).kurt()

df_temp6 = df_temp6.fillna(0)

In [16]:
df_temp7 = df_temp6

In [17]:
var_lst = ['lnRroll5','turnoverRroll5','lnRroll20','stdlnRroll20','sharpe20','skewlnR60','kurtlnR60']
col_lst = list(df_temp6.columns)
for var in var_lst:
    for column in col_lst:
        if (var == column) or (column in var_lst):
            continue
        else:
            column_name = var + '_' + column
            df_temp7[column_name] = df_temp6[var]*df_temp6[column]

In [18]:
df_temp6.to_csv('C:/Users/jxjsj/Desktop/JupyterHome/Data/SZCZ_lnr_rv_w_m_ntd_080101-190131adj.csv')